In [1]:
import geopandas as gpd
import pandas as pd
import verde as vd
import numpy as np
from shapely import geometry

import pyproj


import my_funcs as mf

import matplotlib.pyplot as plt
%matplotlib widget

gdb='/home/ggrl/geodatabase/'

In [5]:
g_3022 = pd.read_csv(gdb+'geof/g3022.csv')
m_3022 = pd.read_csv(gdb+'geof/m3022.csv')

In [6]:
geof_1039 = pd.read_csv(gdb+'geof/geof_1039.csv')

In [10]:
g_1089 = pd.read_csv(gdb+'geof/g1089')
m_1089 = pd.read_csv(gdb+'geof/m1089')

In [11]:
grids_1089_50k = gpd.read_file('/home/ggrl/Desktop/1089_area_50k.shp')

In [15]:
grids_1089_50k.head(10)

,field_1,id_folha,region,geometry
0,23129,SC23_Y_B_III_4,"(-45.25, -45.0, -10.5, -10.25)","POLYGON ((-45.25000 -10.25000, -45.00000 -10.2..."
1,23130,SC23_Y_B_VI_2,"(-45.25, -45.0, -10.75, -10.5)","POLYGON ((-45.25000 -10.50000, -45.00000 -10.5..."
2,23131,SC23_Y_B_VI_4,"(-45.25, -45.0, -11.0, -10.75)","POLYGON ((-45.25000 -10.75000, -45.00000 -10.7..."
3,23132,SC23_Y_D_III_2,"(-45.25, -45.0, -11.25, -11.0)","POLYGON ((-45.25000 -11.00000, -45.00000 -11.0..."
4,23305,SC23_Z_A_I_3,"(-45.0, -44.75, -10.5, -10.25)","POLYGON ((-45.00000 -10.25000, -44.75000 -10.2..."
5,23306,SC23_Z_A_IV_1,"(-45.0, -44.75, -10.75, -10.5)","POLYGON ((-45.00000 -10.50000, -44.75000 -10.5..."
6,23307,SC23_Z_A_IV_3,"(-45.0, -44.75, -11.0, -10.75)","POLYGON ((-45.00000 -10.75000, -44.75000 -10.7..."
7,23308,SC23_Z_C_I_1,"(-45.0, -44.75, -11.25, -11.0)","POLYGON ((-45.00000 -11.00000, -44.75000 -11.0..."
8,23480,SC23_Z_A_I_2,"(-44.75, -44.5, -10.25, -10.0)","POLYGON ((-44.75000 -10.00000, -44.50000 -10.0..."
9,23481,SC23_Z_A_I_4,"(-44.75, -44.5, -10.5, -10.25)","POLYGON ((-44.75000 -10.25000, -44.50000 -10.2..."


In [14]:
grids_1089_50k.plot('id_folha')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [16]:
for index, row in grids_1089_50k.iterrows():

    region = grids_1089_50k.region

    data = g_1089[vd.inside((g_1089.LONG, g_1089.LAT), region = region)]

    coordinates = (mag_SC23_Z_C_I.X.values, mag_SC23_Z_C_I.Y.values)


    chain = vd.Chain([
            ('trend', vd.Trend(degree=1)),
            ('reduce', vd.BlockReduce(np.median, spacing=500)),
            ('spline', vd.Spline())
        ])
        
    chain.fit(coordinates, mag_SC23_Z_C_I.MAGIGRF)

    cv     = vd.BlockKFold(spacing=499,
                        n_splits=1,
                        shuffle=True)

    scores = vd.cross_val_score(chain,
                                coordinates,
                                mag_SC23_Z_C_I.MAGIGRF,
                                cv=cv)


    grid_mag = chain.grid(spacing=100, data_names='MAGIGRF',pixel_register=True)
    grids_mag= vd.distance_mask(coordinates, maxdist=499, grid= grid_mag)
    print(region)

0     (-45.25, -45.0, -10.5, -10.25)
1     (-45.25, -45.0, -10.75, -10.5)
2     (-45.25, -45.0, -11.0, -10.75)
3     (-45.25, -45.0, -11.25, -11.0)
4     (-45.0, -44.75, -10.5, -10.25)
5     (-45.0, -44.75, -10.75, -10.5)
6     (-45.0, -44.75, -11.0, -10.75)
7     (-45.0, -44.75, -11.25, -11.0)
8     (-44.75, -44.5, -10.25, -10.0)
9     (-44.75, -44.5, -10.5, -10.25)
10    (-44.75, -44.5, -10.75, -10.5)
11    (-44.75, -44.5, -11.0, -10.75)
12     (-44.5, -44.25, -10.0, -9.75)
13    (-44.5, -44.25, -10.25, -10.0)
14    (-44.5, -44.25, -10.5, -10.25)
15    (-44.5, -44.25, -10.75, -10.5)
16    (-44.5, -44.25, -11.0, -10.75)
17     (-44.25, -44.0, -10.0, -9.75)
18    (-44.25, -44.0, -10.25, -10.0)
19    (-44.25, -44.0, -10.5, -10.25)
20    (-44.25, -44.0, -10.75, -10.5)
21    (-44.25, -44.0, -11.0, -10.75)
22      (-44.0, -43.75, -9.75, -9.5)
23     (-44.0, -43.75, -10.0, -9.75)
24    (-44.0, -43.75, -10.25, -10.0)
25    (-44.0, -43.75, -10.5, -10.25)
26    (-44.0, -43.75, -10.75, -10.5)
2

In [15]:
plt.figure()
grids_mag.MAGIGRF.plot(cmap='hsv',vmin=-400,vmax=400)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500001.8, 554701.8100000002, 8783977.48, 8839304.829999998)

In [31]:
chain_list = ['CTCOR', 'KPERC', 'MDT', 'eU', 'eth']

grids = {'CTCOR':(),'KPERC':(), 'MDT':(), 'eU':(), 'eth':()}

scores = {'CTCOR':(),'KPERC':(), 'MDT':(), 'eU':(), 'eth':()}


coordinates = (gama_SC23_Z_C_I.X.values, gama_SC23_Z_C_I.Y.values)

for i in chain_list:
    print(i)
    
    chain = vd.Chain([
        ('trend', vd.Trend(degree=1)),
        ('reduce', vd.BlockReduce(np.median, spacing=500)),
        ('spline', vd.Spline())
    ])
    
    chain.fit(coordinates, gama_SC23_Z_C_I[i])
    '''
    cv     = vd.BlockKFold(spacing=1000,
                  n_splits=1,
                  shuffle=True)

    scores[i] = vd.cross_val_score(chain,
                             coordinates,
                             geof_SF23_Y_A_VI_3_DEM[i],
                             cv=cv)
    '''
    grid = chain.grid(spacing=100, data_names=[i],pixel_register=True)
    grids[i] = vd.distance_mask(coordinates, maxdist=500, grid= grid)


CTCOR
KPERC
MDT
eU
eth


In [37]:
plt.figure(figsize=(9,9))
grids['eU'].eU.plot(cmap='hsv',vmin=0)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(500005.30000000005, 554693.8600000001, 8783985.229999999, 8839303.77)

In [169]:
plt.figure(figsize=(9,9))
grids['MAGR'].MAGR.plot(cmap='hsv')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(294395.0, 320223.0, 7482805.000000001, 7510694.999999999)

In [170]:
plt.figure()
plt.hist(scores['MDT'],bins='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  0.,  3.,
         2.,  4.,  6., 12., 22., 23., 24.]),
 array([-0.63206263, -0.55239542, -0.47272822, -0.39306101, -0.31339381,
        -0.23372661, -0.1540594 , -0.0743922 ,  0.005275  ,  0.08494221,
         0.16460941,  0.24427661,  0.32394382,  0.40361102,  0.48327823,
         0.56294543,  0.64261263,  0.72227984,  0.80194704,  0.88161424,
         0.96128145]),
 <BarContainer object of 20 artists>)

In [119]:
plt.figure()
plt.hist(scores['MAGR'],bins='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 1.,  0.,  1.,  5.,  4.,  2.,  3.,  6., 13., 14., 51.]),
 array([-1.49912766, -1.27312413, -1.0471206 , -0.82111707, -0.59511354,
        -0.36911001, -0.14310648,  0.08289705,  0.30890058,  0.53490411,
         0.76090764,  0.98691117]),
 <BarContainer object of 11 artists>)

In [120]:
plt.figure(figsize=(9,9))
grids['CTC'].CTC.plot(cmap='hsv')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(294395.0, 320223.0, 7482805.000000001, 7510694.999999999)

In [108]:
plt.figure()
plt.hist(scores['CTC'],bins='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 1.,  0.,  1.,  0.,  0.,  1.,  6.,  8.,  6., 13., 22., 22., 20.]),
 array([-2.03184542, -1.80324847, -1.57465151, -1.34605456, -1.11745761,
        -0.88886065, -0.6602637 , -0.43166674, -0.20306979,  0.02552717,
         0.25412412,  0.48272108,  0.71131803,  0.93991499]),
 <BarContainer object of 13 artists>)

In [161]:
plt.figure(figsize=(9,9))
grids['UC'].UC.plot(cmap='hsv')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(294395.0, 320223.0, 7482805.000000001, 7510694.999999999)

In [162]:
plt.figure()
plt.hist(scores['UC'],bins='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 1.,  1.,  5., 10., 13., 24., 16., 17.,  8.,  4.,  1.]),
 array([-0.47906598, -0.39240964, -0.3057533 , -0.21909697, -0.13244063,
        -0.04578429,  0.04087205,  0.12752839,  0.21418473,  0.30084106,
         0.3874974 ,  0.47415374]),
 <BarContainer object of 11 artists>)

In [128]:
plt.figure(figsize=(9,9))
grids['THC'].THC.plot(cmap='hsv')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(294395.0, 320223.0, 7482805.000000001, 7510694.999999999)

In [101]:
plt.figure()
plt.hist(scores['THC'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 2.,  0.,  0.,  0.,  0.,  0., 15., 27., 35., 21.]),
 array([-2.2078012 , -1.89833042, -1.58885963, -1.27938885, -0.96991806,
        -0.66044728, -0.35097649, -0.04150571,  0.26796507,  0.57743586,
         0.88690664]),
 <BarContainer object of 10 artists>)

In [130]:
plt.figure(figsize=(9,9))
grids['KC'].KC.plot(cmap='hsv',vmin=0)
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(294395.0, 320223.0, 7482805.000000001, 7510694.999999999)

In [107]:
plt.figure()
plt.hist(scores['KC'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 3.,  1.,  4.,  1., 10.,  9., 16., 19., 25., 12.]),
 array([-0.50901309, -0.36927396, -0.22953483, -0.08979571,  0.04994342,
         0.18968254,  0.32942167,  0.46916079,  0.60889992,  0.74863905,
         0.88837817]),
 <BarContainer object of 10 artists>)

In [96]:
chain = vd.Chain([
        ('trend', vd.Trend(degree=3)),
        ('reduce', vd.BlockReduce(np.median, spacing=1000)),
        ('spline', vd.Spline())
    ])
    
chain.fit(coordinates, geof_SF23_Y_A_VI_3_DEM.UC)

cv     = vd.BlockKFold(spacing=1000,
                  n_splits=100,
                  shuffle=True)

scores_uc = vd.cross_val_score(chain,
                            coordinates,
                            geof_SF23_Y_A_VI_3_DEM.UC,
                            cv=cv)
    
grid_1 = chain.grid(spacing=200, data_names='UC',pixel_register=True)
grid_uc = vd.distance_mask(coordinates, maxdist=1000, grid= grid_1)

In [163]:
plt.figure()
grid_uc.UC.plot(cmap='hsv')
plt.axis('scaled')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(294395.0, 320223.0, 7482805.000000001, 7510694.999999999)

In [164]:
plt.figure()
plt.hist(scores_uc,bins='auto')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array([ 4.,  5.,  5., 13., 20., 22., 16.,  7.,  5.,  3.]),
 array([-0.42350206, -0.33660169, -0.24970132, -0.16280095, -0.07590058,
         0.01099979,  0.09790016,  0.18480053,  0.2717009 ,  0.35860127,
         0.44550164]),
 <BarContainer object of 10 artists>)

In [33]:
data = list()
for i in chain_list:
    df = grids[i].to_dataframe()
    data.append(df[i])

In [34]:
geof = pd.concat(data,axis=1, join='inner')

geof.reset_index(inplace=True)
geof

,easting,northing,CTCOR,KPERC,MDT,eU,eth
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140
...,...,...,...,...,...,...,...
302486,554643.870457,8.838854e+06,1138.891026,0.760344,471.539519,1.726946,12.759650
302487,554643.870457,8.838954e+06,1114.599345,0.783834,466.898206,1.429977,12.531083
302488,554643.870457,8.839054e+06,1088.677543,0.806869,462.777227,1.143662,12.318743
302489,554643.870457,8.839154e+06,1059.884453,0.828355,459.411416,0.881483,12.125568


In [40]:
geof['geometry'] = [geometry.Point(x,y) for x, y in zip(geof['easting'], geof['northing'])]

In [41]:
gdf = gpd.GeoDataFrame(geof,crs=32723)
gdf = gdf.set_crs(32723, allow_override=True)

gdf

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,MAGIGRF
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268,POINT (500055.290 8784035.247),50.464271
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410,POINT (500055.290 8784135.280),50.678373
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627,POINT (500055.290 8784235.314),50.935766
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406,POINT (500055.290 8784335.347),51.153968
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140,POINT (500055.290 8784435.381),51.291826
...,...,...,...,...,...,...,...,...,...
302486,554643.870457,8.838854e+06,1138.891026,0.760344,471.539519,1.726946,12.759650,POINT (554643.870 8838853.619),37.771953
302487,554643.870457,8.838954e+06,1114.599345,0.783834,466.898206,1.429977,12.531083,POINT (554643.870 8838953.653),30.463146
302488,554643.870457,8.839054e+06,1088.677543,0.806869,462.777227,1.143662,12.318743,POINT (554643.870 8839053.686),24.237343
302489,554643.870457,8.839154e+06,1059.884453,0.828355,459.411416,0.881483,12.125568,POINT (554643.870 8839153.720),20.059724


In [26]:
df_mag = grids_mag.to_dataframe()

In [29]:
df_mag.reset_index(inplace=True)

In [30]:
df_mag

,easting,northing,MAGIGRF
0,500051.800009,8.784028e+06,50.464271
1,500051.800009,8.784128e+06,50.678373
2,500051.800009,8.784228e+06,50.935766
3,500051.800009,8.784328e+06,51.153968
4,500051.800009,8.784428e+06,51.291826
...,...,...,...
302486,554651.809991,8.838855e+06,37.771953
302487,554651.809991,8.838955e+06,30.463146
302488,554651.809991,8.839055e+06,24.237343
302489,554651.809991,8.839155e+06,20.059724


In [38]:
geof['MAGIGRF'] = df_mag.MAGIGRF

In [39]:
geof

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,MAGIGRF
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268,POINT (500055.290 8784035.247),50.464271
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410,POINT (500055.290 8784135.280),50.678373
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627,POINT (500055.290 8784235.314),50.935766
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406,POINT (500055.290 8784335.347),51.153968
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140,POINT (500055.290 8784435.381),51.291826
...,...,...,...,...,...,...,...,...,...
302486,554643.870457,8.838854e+06,1138.891026,0.760344,471.539519,1.726946,12.759650,POINT (554643.870 8838853.619),37.771953
302487,554643.870457,8.838954e+06,1114.599345,0.783834,466.898206,1.429977,12.531083,POINT (554643.870 8838953.653),30.463146
302488,554643.870457,8.839054e+06,1088.677543,0.806869,462.777227,1.143662,12.318743,POINT (554643.870 8839053.686),24.237343
302489,554643.870457,8.839154e+06,1059.884453,0.828355,459.411416,0.881483,12.125568,POINT (554643.870 8839153.720),20.059724


In [137]:
gdf.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [42]:
l_100k = mf.litologia('l_100k')

In [43]:
rparaim = mf.mapa(l_100k,'Rio Paraim')

In [44]:
rparaim.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [45]:
rparaim = rparaim.to_crs(32723)
rparaim.crs

<Projected CRS: EPSG:32723>
Name: WGS 84 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: World - S hemisphere - 48°W to 42°W - by country
- bounds: (-48.0, -80.0, -42.0, 0.0)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [49]:
rparaim.reset_index(inplace=True)

In [50]:
rparaim

,index,ID_UNIDADE,SIGLA,HIERARQUIA,NOME,AMBIENTE_T,SUB_AMBIEN,SIGLA_PAI,NOME_PAI,LEGENDA,...,ERA_MIN,ERA_MAX,SISTEMA_MI,SISTEMA_MA,EPOCA_MIN,EPOCA_MAX,SIGLAS_HIS,SHAPE_AREA,SHAPE_LEN,geometry
0,2816,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.000041,0.037400,"POLYGON ((500176.894 8821390.751, 500170.654 8..."
1,2820,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.001087,0.257896,"POLYGON ((505306.762 8803062.104, 505258.182 8..."
2,2821,4873,NP1rp3,Unidade,"Unidade Rio Preto, xisto",None,None,NP1rp,NP1rp - Grupo Rio Preto,Muscovita-xistos e quartzo-xistos predominante...,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"MP3NP1rp3, M3N1rp1x, MP3NP1rp1x",0.000170,0.094417,"POLYGON ((505145.795 8801235.292, 506323.529 8..."
3,2823,4876,NP1rp2q,Litofácies,"Litofácies Rio Preto, metassedimentos quartzos...",None,None,MP3NP1rp2,"MP3NP1rp2 - Unidade Rio Preto, metassedimentos...",Quartzitos a muscovita-quartzitos puros a impu...,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"M3N1rp2q, MP3NP1rp2q",0.000015,0.020423,"POLYGON ((509741.424 8784951.753, 509758.753 8..."
4,2825,5773,PP2cmv,Unidade,"Unidade Cristalândia do Piauí, metamáfica",None,None,A4PPc,A4PPc - Complexo Cristalândia do Piauí,"Corpos gabróicos a metagabros, compostos por a...",...,Paleoproterozóico,Paleoproterozóico,Riaciano,Riaciano,Série 3,Série 3,None,0.000323,0.077072,"POLYGON ((510857.857 8811448.300, 510816.261 8..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,54750,4876,NP1rp2q,Litofácies,"Litofácies Rio Preto, metassedimentos quartzos...",None,None,MP3NP1rp2,"MP3NP1rp2 - Unidade Rio Preto, metassedimentos...",Quartzitos a muscovita-quartzitos puros a impu...,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"M3N1rp2q, MP3NP1rp2q",0.000045,0.063783,"POLYGON ((548787.138 8829535.408, 548759.773 8..."
137,54752,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.012576,0.986684,"POLYGON ((543318.427 8837536.690, 543374.133 8..."
138,54754,1006,N1dl,Não definida,Coberturas detrito-lateríticas ferruginosas,Cobertura superficial,Cobertura clasto-lateritica,None,None,Coberturas de solos residuais argilo-arenosos ...,...,Cenozóico,Cenozóico,Neogeno,Neogeno,Superior,Superior,NQdl,0.043917,1.961410,"POLYGON ((554660.641 8807055.364, 554656.953 8..."
139,54755,4873,NP1rp3,Unidade,"Unidade Rio Preto, xisto",None,None,NP1rp,NP1rp - Grupo Rio Preto,Muscovita-xistos e quartzo-xistos predominante...,...,Neoproterozóico,Neoproterozóico,Toniano,Toniano,Wenloquiano,Wenloquiano,"MP3NP1rp3, M3N1rp1x, MP3NP1rp1x",0.023544,3.651426,"POLYGON ((554702.380 8832876.999, 554702.379 8..."


In [46]:
rparaim.plot('SIGLA')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [51]:
geof['closest_unid'] = gdf['geometry'].apply(lambda x: rparaim['SIGLA'].iloc[rparaim.distance(x).idxmin()]) # .idxmin() Retorna o indice do menor valor 

In [52]:
geof.head()

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,MAGIGRF,closest_unid
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268,POINT (500055.290 8784035.247),50.464271,NP1rp3
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410,POINT (500055.290 8784135.280),50.678373,NP1rp3
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627,POINT (500055.290 8784235.314),50.935766,NP1rp3
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406,POINT (500055.290 8784335.347),51.153968,NP1rp3
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140,POINT (500055.290 8784435.381),51.291826,NP1rp3


In [53]:
# Setting projections

# South America WGS 84 Geográfica
p1 = pyproj.Proj("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

# WGS84 UTM 
p2 = pyproj.Proj("+proj=utm +zone=23 +south +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

# 1105
geof['LONGITUDE'], geof['LATITUDE'] = pyproj.transform(p2, p1,
                                                geof.easting.values,
                                                geof.northing.values)

<ipython-input-53-229dbd804a89>:10: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  geof['LONGITUDE'], geof['LATITUDE'] = pyproj.transform(p2, p1,


In [54]:
geof.head()

,easting,northing,CTCOR,KPERC,MDT,eU,eth,geometry,MAGIGRF,closest_unid,LONGITUDE,LATITUDE
0,500055.289543,8.784035e+06,1708.875338,1.685567,508.936948,1.907549,17.543268,POINT (500055.290 8784035.247),50.464271,NP1rp3,-44.999494,-10.999867
1,500055.289543,8.784135e+06,1712.006845,1.627513,508.831103,1.922783,17.808410,POINT (500055.290 8784135.280),50.678373,NP1rp3,-44.999494,-10.998963
2,500055.289543,8.784235e+06,1693.736960,1.561761,509.035380,1.935492,17.728627,POINT (500055.290 8784235.314),50.935766,NP1rp3,-44.999494,-10.998058
3,500055.289543,8.784335e+06,1639.938707,1.493374,510.118943,1.950659,17.013406,POINT (500055.290 8784335.347),51.153968,NP1rp3,-44.999494,-10.997153
4,500055.289543,8.784435e+06,1561.172489,1.426346,511.533855,1.962516,15.848140,POINT (500055.290 8784435.381),51.291826,NP1rp3,-44.999494,-10.996248


In [55]:
geof.to_csv(gdb+'OI_SC23_Z_C_I.csv',index=False)